In [1]:
import pandas as pd
import datetime 
import numpy as np
import pickle
import matplotlib.pyplot as plt
import dtale
import re

In [2]:
smileon = pd.read_csv('../data/smileon_edited.csv')

In [3]:
pd.set_option('display.max_columns', 118)
smileon.head()

,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,0,3638169,93115324,registration,9/30/2021 16:56,9/30/2021 16:57,1,123747,123747,9/30/2021 16:57,No,NaN,1960,Smyrna,Rutherford,TN,37167.0,1.0,NaN,English,African American,Female,$0 - $21 280,Other,SNAP; SSI/SSDI,NaN,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,Yes,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4431194,93115188,registration,9/30/2021 16:53,9/30/2021 16:53,0,123747,123747,9/30/2021 16:53,No,NaN,1959,Nashville,Davidson,TN,37211.0,1.0,NaN,English,Caucasian,Female,$0 - $21 280,Dental Clinic Staff,NaN,More than 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,NaN,Referral Follow-Up,Yes,Interfaith Dental Clinic - Nashville,Yes,NaN,No,Yes,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2941341,93115081,clinical,9/30/2021 16:50,9/30/2021 16:51,1,161979,161979,9/30/2021 16:51,No,NaN,1942,Memphis,Shelby,TN,38104.0,1.0,NaN,NaN,Asian,Male,$0 - $21 280,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,NaN,Continued with Navigation,NaN,NaN,10/8/2019,Christ Community Health Services- Broad Avenue,Provider Site,No,Christ Community Health Services- Broad Avenue,Yes,NaN,NaN,No,Good,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/8/2019,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274 - Bitewings - four films - 5; 1110 - Proph...,No,Yes,No
3,3,3823933,93115032,clinical,9/30/2021 16:48,9/30/2021 16:49,1,161979,161979,9/30/2021 16:49,No,NaN,1960,Memphis,Shelby,TN,38109.0,1.0,NaN,NaN,African American,Male,$0 - $21 280,Dental Clinic Staff,NaN,More than 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,11/10/2020,Christ Community Health Services- Broad Avenue,

In [4]:
smileon = smileon.sort_values(['patient_id', 'entrydate'])

In [5]:
smileon = smileon.reset_index()
smileon.head(70)

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,13186,13186,-2,84708447,registration,2/19/2021 12:10,2/19/2021 12:56,46,119513,119513,2/19/2021 12:56,No,NaN,1954,Johnson City,Washington,TN,37604.0,4.0,NaN,Spanish,Hispanic,Female,$0 - $21 280,Dental Clinic Staff,NaN,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Northeast Tennessee Community Health Centers I...,Provider Site,Yes,Northeast Tennessee Community Health Centers I...,Yes,NaN,NaN,No,Dont know,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47543,47543,-2,62945363,clinical,5/15/2019 10:00,5/15/2019 11:00,60,119473,119473,5/15/2019 10:05,No,NaN,1953,NaN,NaN,NaN,NaN,2,NaN,NaN,Caucasian,Male,$32921-$37035,Dental Clinic Staff,NaN,Less than 12 months,No,No,NaN,No further questions from client,NaN,NaN,NaN,Healing Hands Health Center - Bristol,NaN,No,NaN,NaN,NaN,NaN,No,Dont know,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
2,42846,42846,-2,65438351,registration,8/13/2019 8:00,8/13/2019 8:05,5,133281,133281,8/13/2019 8:05,No,NaN,1957,Memphis,Shelby,TN,38127.0,1,NaN,English,African American,Female,$0 - $21 280,Dental Clinic Staff,NaN,Less than 12 months,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Christ Community Health Services- Raleigh,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dental cost; No insurance; Fear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,228,228,-2,92968098,clinical,9/27/2021 12:56,9/27/2021 12:56,0,140162,140162,9/27/2021 12:56,No,NaN,1957,Mount Pleasant,Maury,TN,NaN,4.0,NaN,NaN,Caucasian,Female,$0 - $21 280,Dental Clinic Staff,SSI/SSDI,3 to 5 years,No,Yes,NaN,NaN,No,Yes,9/27/2021,TDH - Maury Co,Provider Site,Yes,NaN,NaN,NaN,NaN,Yes,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
#dropping the first 69 rows (rows 0-68) because they're weird - looks like test entries or something?  
#all data for patient's # -1 and -2
smileon = smileon.drop(range(0,69))
smileon = smileon.reset_index(drop=True)
smileon.head(10)

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,32182,32182,2384106,70281146,clinical,1/20/2020 13:25,1/20/2020 13:25,0,134549,134549,1/27/2020 13:25,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,Yes,Yes,Yes
1,53004,53004,2384106,59821075,clinical,1/9/2019 10:07,1/9/2019 10:07,0,118821,118821,1/28/2019 10:07,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,NaN,Yes,NaN
2,34169,34169,2384106,69642939,clinical,12/16/2019 10:35,12/16/2019 10:35,0,134549,134549,1/6/2020 10:35,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,NaN,NaN,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,No,Yes,No
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:59,10/9/2018 18:59,0,118821,118821,10/11/2018 18:59,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental

In [7]:
smileon['patient_id'].value_counts()

3212379    54
2441776    43
2758373    42
2431132    37
2416076    35
           ..
2628473     1
2530943     1
2511760     1
2635370     1
2823981     1
Name: patient_id, Length: 10709, dtype: int64

In [8]:
#NOTE :  THERE ARE 10709 UNIQUE PATIENTS IN THIS DF

In [9]:
smileon['reportversion'].value_counts()

clinical        40173
registration    17618
Name: reportversion, dtype: int64

In [10]:
#NOTE: BUT THERE ARE 17622 REGISTRATIONS.  MEANINS SOME PATIENTS, THE SAME PATIENT EVEN, REGISTERED MORE THAN ONCE

In [11]:
#I would really like to be able to see all the columns.  Going back to fix that
smileon.loc[smileon['patient_id']==2416076]

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
2477,32756,32756,2416076,70080607,clinical,1/13/2020 12:49,1/13/2020 12:52,3,117268,117268,1/20/2020 12:52,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nashville,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Poor,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272 - Bitewings - two films - 5; 1110 - Prophy...,No,Yes,No
2478,33854,33854,2416076,69737230,clinical,12/19/2019 11:37,12/19/2019 11:38,1,116749,116749,1/8/2020 11:38,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nashville,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Poor,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2330 - Resin one surface - Anterior - 15,No,Yes,Yes
2479,55777,55777,2416076,57225628,clinical,10/17/2018 10:15,10/17/2018 10:16,1,118821,118821,10/18/2018 10:16,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nashville,NaN,No,Interfaith Dental Clinic - Nashville,Yes,No,NaN,Yes,Poor,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5221 - Partial denture - resin base - immediat...,NaN,NaN,NaN
2480,56291,56291,2416076,56828901,clinical,10/2/2018 13:47,10/2/2018 13:49,2,118821,118821,10/4/2018 13:49,No,NaN,1945,Madison,Davidson,TN,37115.0,1,NaN,NaN,Caucasian,Male,NaN,This encounter,NaN,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,NaN,Interfaith Dental Clinic - Nas

In [12]:
#my df did not organize quite as well as i'd hoped - try again?
smileon = smileon.sort_values(['patient_id', 'callreportnum'])

#So, i was hoping that i'd see a row of needed procedures and then see them drop off over time to show progress ... but that's not what i have here.  What i have is procedures that were done.  And, i guess, how many times they were done.  Like oce_2330 - that's "resin one surface, anterior".  and sometimes it's in this patient's row x1, others x6.  so - i'm guessing that means they had six teeth done, with resin applied to one surface per tooth.

so, i can see that, with each visit, the patient had more work done.  and in the end, their plan was not completed and their full function was not restored.  Does that mean they weren't served ... ?
Obviously - no.  they were in better condition when they left than when they arrived - we see a ton of work done

Number of procedures per patient is a poor metric of success.  some need more help than others.  

perhaps a window could be - how many of these procedures are being offered overall.  a sum of each column.  finding the maximum column.  totalling some of them together since they're basically the same thing - veneers for example, irresespective of how many sides of the tooth.  that's important for billing - but not for treatment stats.

another window could be what complaint a person has when they arrive, and what treatments they received.  That will be harder to determine - but could be interesting.  Let's take a look at the complaints column.

In [13]:
smileon.head()

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,Reviewed,ReferralsMade,patient_bday,city,county,state,zip,demographic_household,demographic_lang,demographic_preferredlang,demographic_ethnic,demographic_gender,demographic_income,demographic_howyouhearprogram,demographic_otherassistance,demographic_lastdentistvisit,pe_insurance,pe_smileonprequalify,pe_nextstep,pe_nextstep2,ve_transitarranged,ve_eligibility,ve_verification_date,si_clinicname,si_typewhereclientenrolled,blq_troubleeating,blq_dentalclinic,blq_patientreferralcommitment,blq_did_needhelp,blq_do_needhelp,blq_qualityoflife,blq_mouthcondition,blq_nextstep,blq_nextstep2,s_num_lowerteeth,s_num_upperteeth,s_dentures,s_dentureswhileeating,s_problems12months,s_brushing,s_needforcare,s_rootfrags,s_lesions,s_urgency,s_decay,s_lastscreening_date,ci_lastseen_date,ci_apptmade,ci_transitprovided,cmi_clinic,cmi_transitproblem,cmi_findingcareproblem,cmi_clientoutcome,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2393,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2950,oce_2954,oce_2980,oce_3220,oce_4341,oce_4342,oce_6930,oce_6980,oce_7111,oce_7140,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7320,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
11,57818,57818,2384106,55083365,registration,8/1/2018 13:22,8/1/2018 14:09,47,118833,118833,7/31/2018 14:09,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,NaN,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,Continued with Navigation,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,Interfaith Dental Clinic - Murfreesboro,Yes,NaN,No,No,Poor,NaN,Referred to dental clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,57743,57743,2384106,55264053,clinical,8/7/2018 10:53,8/7/2018 10:55,2,116749,116749,8/7/2018 10:55,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,Yes,8/1/2018,NaN,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,9.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,Once daily,Yes,Yes,NaN,Urgent Care - within next week-pain or infection,Yes,8/1/2018,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,57851,57851,2384106,55628771,clinical,8/1/2018 8:53,8/1/2018 8:54,1,118821,118821,8/21/2018 8:54,No,NaN,1944,Murfreesboro,Rutherford,TN,37130.0,5,NaN,Arabic,Unknown,Female,$0 - $21 280,Dental Clinic Staff,SNAP; TennCare,3 to 5 years,No,Yes,NaN,Continued with Navigation,NaN,NaN,8/1/2018,Interfaith Dental Clinic - Murfreesboro,NaN,Yes,NaN,Yes,NaN,NaN,No,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2018,NaN,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,Dental cost,Navigation to dental clinic for dental exam an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7140 - Extraction - erupted tooth or exposed r...,NaN,NaN,NaN
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:5

In [14]:
smileon['s_problems12months'].value_counts()

Difficulty when biting or chewing foods                                                                                                                                                468
Experienced pain or discomfort                                                                                                                                                         381
Difficulty when biting or chewing foods; Experienced pain or discomfort                                                                                                                362
Dry mouth                                                                                                                                                                              344
Difficulty when biting or chewing foods; Difficulty with speech or pronouncing words; Dry mouth; Felt anxiety; Felt embarrassement; Avoided smiling; Experienced pain or discomfort    317
                                                                 

#okay - they warned about that.  there's only 7 options for entry, but they're not broken up here.  There's a lot of possible combos.  

so, i need to break this up into 7 columns, and populate each with a 1 if that phrase exists in the 12months complaints column.  probably a regex function. they do at least show in order ... so, if a person is having problems biting foods, that will always be first in the list.

In [15]:
#let's see about some stats:
smileon.describe()

,index,Unnamed: 0,patient_id,callreportnum,calllength,callstaff_id,entrystaff_id,zip,s_num_lowerteeth,s_num_upperteeth,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2954,oce_2980,oce_3220,oce_4341,oce_6930,oce_6980,oce_7111,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7286,oce_7310,oce_7311,oce_7321,oce_7410,oce_7411,oce_7471,oce_7510,oce_7971,oce_7972,oce_9110,oce_9951
count,57791.000000,57791.000000,5.779100e+04,5.779100e+04,57791.000000,57791.000000,57791.000000,54787.000000,6183.000000,5299.000000,1337.000000,12.000000,891.000000,50.000000,104.000000,46.000000,10.0,944.000000,940.000000,579.000000,428.000000,8.0,1226.000000,1341.000000,171.000000,2.0,134.000000,19.000000,74.000000,15.0,3.0,7.0,1396.000000,12.000000,2.0,9.000000,819.000000,5.000000,6.000000,19.000000,3.000000,486.000000,1.0,68.000000,225.000000,30.000000,80.000000,16.000000,1.0,4.0,12.0,2.0,1.0,26.000000,83.0
mean,28920.357997,28920.357997,3.083800e+06,7.417992e+07,4.097541,124761.271201,124851.273036,37551.853852,10.084910,9.824495,2.549738,2.416667,3.244669,1.280000,1.057692,1.043478,1.0,1.507415,1.361702,1.255613,1.343458,1.0,1.273246,1.158091,1.116959,1.0,1.037313,1.052632,1.108108,1.0,1.0,1.0,1.832378,1.083333,1.0,1.777778,1.800977,1.200000,1.166667,3.000000,3.666667,2.425926,1.0,1.073529,1.853333,1.233333,1.237500,2.187500,1.0,1.5,1.0,1.0,1.0,1.038462,1.0
std,16707.115214,16707.115214,5.808006e+05,1.086729e+07,10.497058,14704.883192,14248.340910,507.528006,3.183853,3.537183,2.173077,2.353270,3.039194,1.050559,0.234290,0.206185,0.0,1.027691,1.062990,0.628297,0.769001,0.0,0.579558,0.524394,0.403374,0.0,0.190240,0.229416,0.424186,0.0,0.0,0.0,0.814200,0.288675,0.0,1.394433,2.092763,0.447214,0.408248,4.558265,3.785939,2.378960,NaN,0.262944,0.931013,0.568321,0.483755,1.833712,NaN,1.0,0.0,0.0,NaN,0.196116,0.0
min,0.000000,0.000000,2.384106e+06,5.508336e+07,0.000000,-1.000000,60132.000000,24201.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
25%,14452.500000,14452.500000,2.633565e+06,6.540958e+07,1.000000,118845.000000,118845.000000,37146.000000,8.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.500000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
50%,28901.000000,28901.000000,2.891697e+06,7.157117e+07,1.000000,123747.000000,123747.000000,37334.000000,10.000000,10.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,2.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.0,1.000000,2.000000,1.000000,1.000000,1.500000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
75%,43409.500000,43409.500000,3.498203e+06,8.423406e+07,3.000000,134546.000000,134546.000000,38018.000000,12.000000,13.000000,3.000000,2.500000,4.000000,1.000000,1.000000,1.000000,1.0,2.000000,2.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,2.000000,1.000000,1.0,2.000000,2.000000,1.000000,1.000000,2.500000,5.000000,3.000000,1.0,1.000000,2.000000,1.000000,1.000000,3.000000,1.0,1.5,1.0,1.0,1.0,1.000000,1.0
max,57859.000000,57859.000000,4.581581e+06,9.350847e+07,735.000000,164262.000000,164262.000000,38654.000000,16.000000,16.000000,17.000000,8.000000,22.000000,8.000000,2.000000,2.000000,1.0,10.000000,27.000000,8.0000

oce_230:   x-rays          1337
oce_1351:  tooth sealant   12
oce_1354:  topical med     891
oce_2140:  filling 1s      50
oce_2150:  filling 2s      104
oce_2160:  filling 3s      46
oce_2161:  filling 4s      10
        TOTAL FILLINGS:        210
oce_2330:  resin 1s        944
oce_2331:  resin 2s        940
oce_2332:  resin 3s        579
oce_2335:  resin 4s        428
oce_2390:  crown           8
oce_2391:  postresin 1s    1226
oce_2392:  postresin 2s    1341
oce_2394:  postresin 3s    171
oce_2915:  prosthetic cast 2
oce_2920:  pros. crown     134
oce_2933:  steel crown     19
oce_2940:  sed. filling    74
oce_2954:  toothrebuild    15
oce_2980:  crown repair    3
oce_3220:  save rot.tooth  7
oce_4341:  plaque removal  1396
oce_6930:  recement bridge 12
oce_6980:  repair bridge   2
oce_7111:  extraction rem. 9
oce_7210:  surg. extract   819
oce_7220:  removal impacts 5
oce_7230:  removal impactpb 6
oce_7240:  removal impactcb 19
oce_7241:  removal complica 3
oce_7250:  surgery roots   486
oce_7261:  sinus closure   1
oce_7286:  biopsy          68
oce_7310:  alveoloplasty   225
oce_7311:  alveoloplasty   30
oce_7321:  alveoloplasty   80
oce_7410:  excision b.lesion 16
oce_7411:  excision b.lesion 1
oce_7471:  removal exostosis 4
oce_7510:  abscess drain   12
oce_7971:  pericoronal gin.  2
oce_7972:  surg. fibroustub. 1
oce_9110:  emergency treatm. 26
oce_9951:  occlusal adjust.  83

TOTAL cosmetic RESINS/RESTORATIONS:  5637
TOTAL SURGERIES:            1696

most common treatment: resin     2738
#2 most common:  plaque removal  1396
#3 most common:  xrays           1337

In [16]:
smileon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57791 entries, 11 to 57790
Columns: 118 entries, index to tp_clientrestored
dtypes: float64(48), int64(7), object(63)
memory usage: 52.5+ MB


In [17]:
#okay - let's figure out how to break apart that complaints column into 7 different columns, one for each:
#s_problems12months
s_difficulty_eating = []
s_difficulty_speaking = []
s_drymouth = []
s_felt_anxiety = []
s_felt_embarrassed = []
s_avoided_smiling = []
s_pain_discomfort = []

In [18]:
smileon[smileon['s_problems12months'].str.contains(r'(\w*\s+)chewing(\s+\w*)')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [19]:
#looks like i can't filter this column yet because of the nan values.  so - can i drop them, then maybe merge things back in?
smileon_drop = smileon.dropna(subset=['s_problems12months'])

In [34]:
regex1 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)chewing(\s+\w*)')]  #3385
regex2 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)speech(\s+\w*)')]  #1348
regex3 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)Dry(\s+\w*)')]  #1454
regex4 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)anxiety(\S+\w*)')]  #1447
regex5 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)embarrassement(\S+\w*)')]  #1894
regex6 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)smiling(\S+\w*)')]  #1394
regex7 = smileon_drop[smileon_drop['s_problems12months'].str.contains(r'(\w*\s+)pain(\s+\w*)')]  #2835

In [ ]:
#okay! that's working - now how to get these columns to populate the empty libraries with "1" or "0" values ... 

In [26]:
#let's play with this dtale tool that robert found!  
d=dtale.show(smileon)
d.open_browser()